In [ ]:
from typing import List

import math
import json
import itertools

import numpy as np
from colormath.color_objects import sRGBColor, HSVColor, LabColor
from colormath.color_conversions import convert_color
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
import plotly.graph_objects as go

from IPython.display import HTML

In [ ]:
class Skin:
    def __init__(self, id, title, date, category_id, r, g, b):
        self.id = id
        self.title = title
        self.date = date
        self.category_id = category_id
        self.original_rgb = {
            'r': r,
            'g': g,
            'b': b,
        }
        self.rgb = sRGBColor(r, g, b, True)
        self.hsv = convert_color(self.rgb, HSVColor)
        self.lab = convert_color(self.rgb, LabColor)
    
    def to_json(self):
        return {
            'id': self.id,
            'title': self.title,
            'date': self.date,
            'category_id': self.category_id,
            'r': self.original_rgb['r'],
            'g': self.original_rgb['g'],
            'b': self.original_rgb['b'],
        }

class Cluster:
    def __init__(self, skins: List[Skin], center: LabColor):
        self.skins = skins
        self.center = center
        self.skin_closest_to_center = self.pick_skin_closest_to_center()
        self.most_saturated_skin = self.pick_most_saturated_skin()
    
    def pick_skin_closest_to_center(self) -> Skin:
        coordinates = [[skin.lab.lab_l, skin.lab.lab_a, skin.lab.lab_b]
                       for skin
                       in self.skins]
        distances = euclidean_distances(
            coordinates,
            [[self.center.lab_l, self.center.lab_a, self.center.lab_b]]
        )[:, 0]
        closest_index = np.argmin(distances)
        return self.skins[closest_index]
    
    def pick_most_saturated_skin(self) -> Skin:
        return sorted(self.skins, key=lambda skin: skin.hsv.hsv_s, reverse=True)[0]
    
    # Set skin_with_neutral_color and skin_with_distinct_color
    def pick_skins_relative_to_other_clusters(self, other_clusters: List['Cluster']):
        skin_coordinates = [[skin.lab.lab_l, skin.lab.lab_a, skin.lab.lab_b]
                            for skin
                            in self.skins]
        center_coordinates = [[cluster.center.lab_l, cluster.center.lab_a, cluster.center.lab_b]
                              for cluster
                              in other_clusters]
        # The list of the sum of the distances between a single skin and other cluster's centers
        distances = np.sum(euclidean_distances(skin_coordinates, center_coordinates), axis=1)
        
        self.skin_with_neutral_color = self.skins[np.argmin(distances)]
        self.skin_with_distinct_color = self.skins[np.argmax(distances)]

In [ ]:
picked_color_num = 6
kmeans_random_state = 3

In [ ]:
def import_skins():
    with open('../input/colors.json') as f:
        colors = json.loads(f.read())
        skins = [Skin(color['id'], color['title'], color['date'], color['category_id'], color['r'], color['g'], color['b']) for color in colors]
    return skins

def export_clusters(clusters: List[Cluster]):
    with open('../output/clusters.json', 'w+') as f:
        f.write(json.dumps([[skin.to_json() for skin in cluster.skins] for cluster in clusters], ensure_ascii=False))

def export_picked_skins(title: str, skins: List[Skin]):
    with open(f'../output/{title}.json', 'w+') as f:
        f.write(json.dumps([skin.to_json() for skin in skins], ensure_ascii=False))

In [ ]:
# Filter out skins with a low saturation
def filter_skins(skins: List[Skin]) -> List[Skin]:
    # return skins
    return list(filter(
        lambda skin: skin.hsv.hsv_s > 0.3 and skin.hsv.hsv_v > 0.3,
        skins
    ))

def clusterize_skins(skins: List[Skin], cluster_count: int) -> List[Cluster]:
    matrix = np.array([[skin.lab.lab_l, skin.lab.lab_a, skin.lab.lab_b] for skin in skins])
    kmeans = KMeans(n_clusters=picked_color_num, random_state=kmeans_random_state).fit(matrix)
    
    # Create a list of skins for each cluster
    skin_lists = [[] for _ in range(cluster_count)]
    for i, skin in enumerate(skins):
        skin_lists[kmeans.labels_[i]].append(skin)
    
    clusters = [Cluster(
                    skin_lists[i],
                    LabColor(kmeans.cluster_centers_[i][0], kmeans.cluster_centers_[i][1], kmeans.cluster_centers_[i][2])
                )
                for i
                in range(cluster_count)]
    
    # Once all the clusters are defined,each cluster can pick the most neutral and distinct color
    # in comparison to other clusters
    for i, cluster in enumerate(clusters):
        other_clusters = clusters[:i] + clusters[i+1:]
        cluster.pick_skins_relative_to_other_clusters(other_clusters)
    
    return clusters

# Order the skins so that adjacent skins have distinct colors
def arrange_skins(skins: List[Skin]) -> List[Skin]:
    def calculate_adjacent_distance_product(skins: List[Skin]) -> float:
        shifted_skins = skins[1:len(skins)] + [skins[0]]
        distances = [euclidean_distances(
                         [[skins[i].lab.lab_l, skins[i].lab.lab_a, skins[i].lab.lab_b]],
                         [[shifted_skins[i].lab.lab_l, shifted_skins[i].lab.lab_a, shifted_skins[i].lab.lab_b]],
                     )[0, 0]
                     for i
                     in range(len(skins))]
        return np.prod(distances)
    
    best_order = []
    max_distance_product = -1
    
    # Fix the first skin and permutate the others
    permutations = [[skins[0]] + list(permutation)
                    for permutation
                    in itertools.permutations(skins[1:])]
    
    for permutation in permutations:
        distance_product = calculate_adjacent_distance_product(permutation)
        if distance_product > max_distance_product:
            best_order = permutation
            max_distance_product = distance_product
    
    # Place the サンデー's color in the second place
    # so that it comes to the top right position in the icon
    sunday_index = -1
    for i, skin in enumerate(best_order):
        if 'サンデー' in skin.title:
            sunday_index = i
            break
    if sunday_index >= 0: # If サンデー exists
        if sunday_index < 1:
            best_order = best_order[len(best_order) - 1 + sunday_index:] + best_order[:len(best_order) - 1 + sunday_index]
        else:
            best_order = best_order[sunday_index - 1:] + best_order[:sunday_index - 1]
    
    return best_order

In [ ]:
def plot_lab(skins: List[Skin]):
    axis_settings = {
        'title': '',
        'dtick': 10,
        'showticklabels': False,
        'showspikes': False,
        # 'showbackground': False,
        # 'backgroundcolor': '#eee',
    }
    
    l = [skin.lab.lab_l for skin in skins]
    a = [skin.lab.lab_a for skin in skins]
    b = [skin.lab.lab_b for skin in skins]
    marker_colors = [f"rgb({skin.original_rgb['r']}, {skin.original_rgb['g']}, {skin.original_rgb['b']})"
                     for skin
                     in skins]
    
    fig = go.Figure(data=[go.Scatter3d(
        x=l,
        y=a,
        z=b,
        mode='markers',
        marker={
            'size': 8,
            'color': marker_colors,
            'opacity': 1.0,
        },
    )])

    fig.update_layout(
        scene={
            'xaxis': {
                # 'title': 'l',
                **axis_settings,
            },
            'yaxis': {
                # 'title': 'a',
                **axis_settings,
            },
            'zaxis': {
                # 'title': 'b',
                **axis_settings,
            },
            'hovermode': False,
        },
        margin={ 't': 0, 'b': 0, 'l': 0, 'r': 0 },
    )
    fig.show()

In [ ]:
def display_clusters(clusters: List[Cluster]):
    sorted_clusters = sorted(clusters, key=lambda cluster: cluster.center.lab_l, reverse=True)
    
    def generate_box_html(skin):
        r, g, b = skin.original_rgb['r'], skin.original_rgb['g'], skin.original_rgb['b']
        return (
            '<div>'
                f'<div style="width: 40px; height: 40px; background-color: rgba({r}, {g}, {b}, 1)"></div>'
                # f'<p>{skin.title}</p>'
            '</div>'
        )
    html = ''
    for cluster in sorted_clusters:
        html += '<div style="display: flex; flex-wrap: wrap; width: 500px; padding: 5px 0">'
        for skin in cluster.skins:
            html += generate_box_html(skin)
        html += '</div>'
    display(HTML(html))

def display_picked_skins(clusters: List[Cluster]):
    def generate_skin_row(skin):
        r, g, b = skin.original_rgb['r'], skin.original_rgb['g'], skin.original_rgb['b']
        return (
            '<tr>'
                f'<td style="width: 40px; height: 40px; background-color: rgba({r}, {g}, {b}, 1)"></td>'
                f'<td style="vertical-align: middle">{skin.title}</td>'
            '</tr>'
        )
    
    def generate_arranged_skin_group_html(title: str, skins: List[Skin]):
        title_element = f'<h1>{title}</h1>'
        return title_element + '<table>' + ''.join([generate_skin_row(skin) for skin in arrange_skins(skins)]) + '</table>'
    
    html = ''
    
    html += generate_arranged_skin_group_html(
        'Average Colors',
        [cluster.skin_closest_to_center for cluster in clusters],
    )
    html += generate_arranged_skin_group_html(
        'Neutral Colors',
        [cluster.skin_with_neutral_color for cluster in clusters],
    )
    html += generate_arranged_skin_group_html(
        'Distinct Colors',
        [cluster.skin_with_distinct_color for cluster in clusters],
    )
    html += generate_arranged_skin_group_html(
        'Saturated Colors',
        [cluster.most_saturated_skin for cluster in clusters],
    )
    display(HTML(html))

In [ ]:
skins = filter_skins(import_skins())
clusters = clusterize_skins(skins, picked_color_num)
export_clusters(clusters)

# Export three kinds of skin set
export_picked_skins('average-skins', arrange_skins([cluster.skin_closest_to_center for cluster in clusters]))
export_picked_skins('neutral-skins', arrange_skins([cluster.skin_with_neutral_color for cluster in clusters]))
export_picked_skins('distinct-skins', arrange_skins([cluster.skin_with_distinct_color for cluster in clusters]))
export_picked_skins('saturated-skins', arrange_skins([cluster.most_saturated_skin for cluster in clusters]))

In [ ]:
plot_lab(skins)

In [ ]:
display_clusters(clusters)

In [ ]:
display_picked_skins(clusters)